In [ ]:
%%bash
sudo apt-get install gcc-multilib
wget https://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
tar -xzf ta-lib-0.4.0-src.tar.gz
cd ta-lib/
./configure --prefix=/usr
make
sudo make install
pip install ta-lib

In [1]:
%load_ext dotenv
%dotenv -o

import os
os.environ["POLYGON_DATA_DIR"]

'/media/mirror/files.polygon.io'

In [2]:
os.environ["CUSTOM_ASSET_FILES_DIR"]

'/home/jovyan/work/data'

In [6]:
from zipline_polygon_bundle.config import PolygonConfig
from zipline_polygon_bundle import date_to_path, trades_schema, cast_trades, custom_aggs_schema, custom_aggs_partitioning, generate_csv_trades_tables, trades_to_custom_aggs
from zipline_polygon_bundle import get_custom_aggs_dates
from zipline_polygon_bundle import get_ticker_universe

from typing import Iterator, Tuple
import warnings

import pandas as pd
import plotly.graph_objs as go

import pyarrow as pa
from pyarrow import dataset as pa_ds
from pyarrow import compute as pa_compute
from pyarrow import compute as pc
from pyarrow import parquet as pa_parquet

from fsspec.implementations.arrow import ArrowFSWrapper

import datetime
import pandas_market_calendars
import numpy as np

import catboost as ca
from catboost import CatBoostClassifier, Pool

import resource

from itables import show

# import talib
import pandas_ta as ta

import cudf
import cupy
cupy.cuda.get_device_id()

0

In [7]:
if not "POLYGON_TICKERS_DIR" in os.environ:
    os.environ["POLYGON_TICKERS_DIR"]= "/home/jovyan/work/data/tickers"

if not "CUSTOM_ASSET_FILES_DIR" in os.environ:
    os.environ["CUSTOM_ASSET_FILES_DIR"]= "/home/jovyan/work/data"

config = PolygonConfig(
    environ=os.environ,
    calendar_name="NYSE",
    start_date="2018-01-01",
    end_date="2023-12-31",
    agg_time="1min",
)

pd.set_option('display.width', 250)

print(f"{config.minute_aggs_dir=}")
print(f"{config.trades_dir=}")
print(f"{config.custom_aggs_dir=}")
print(f"{config.aggs_dir=}")
print(f"{config.by_ticker_dir=}")

config.minute_aggs_dir='/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/minute_aggs_v1'
config.trades_dir='/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1'
config.custom_aggs_dir='/home/jovyan/work/data/60sec_aggs'
config.aggs_dir='/home/jovyan/work/data/60sec_aggs'
config.by_ticker_dir='/home/jovyan/work/data/60sec_aggs_by_ticker'


In [8]:
ticker_universe = get_ticker_universe(config, fetch_missing=True)
ticker_universe

tickers_csv_path='/home/jovyan/work/data/tickers/tickers_2018-01-01_2023-12-31.csv'
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 49789 entries, ('CpP', 'XNYS', '0000831001', 'PFD', nan, True) to ('ZZrw', 'XNYS', '0000748015', 'RIGHT', nan, False)
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   start_date        49789 non-null  datetime64[ns]
 1   end_date          49789 non-null  datetime64[ns]
 2   last_updated_utc  49789 non-null  object        
 3   name              49789 non-null  object        
 4   share_class_figi  49789 non-null  object        
 5   delisted_utc      49789 non-null  object        
 6   currency_name     49789 non-null  object        
 7   locale            49789 non-null  object        
 8   market            49789 non-null  object        
dtypes: datetime64[ns](2), object(7)
memory usage: 6.3+ MB


,,,,,,start_date,end_date,last_updated_utc,name,share_class_figi,delisted_utc,currency_name,locale,market
ticker,primary_exchange,cik,type,composite_figi,active,,,,,,,,,
CpP,XNYS,0000831001,PFD,NaN,True,2018-01-02,2018-01-02,2024-12-03,[Citigroup Inc],[],[],[usd],[us],[stocks]
XKII,ARCX,0001081019,NaN,BBG00JHNY9Z4,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Intelligent Structures ETF],[BBG00JHNYBP0],[],[usd],[us],[stocks]
XKST,ARCX,0001081019,NaN,BBG00JHP5PG6,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Smart Mobility ETF],[BBG00JHP5Q56],[],[usd],[us],[stocks]
LDRS,ARCX,0001415726,NaN,UNKNOWN,True,2018-01-02,2018-01-02,2024-12-03,[Innovator IBD� ETF Leaders ETF],[UNKNOWN],[],[usd],[us],[stocks]
XKFS,ARCX,0001081019,NaN,BBG00JHP6203,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Future Security ETF],[BBG00JHP62Q5],[],[usd],[us],[stocks]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GOEV,XNAS,0001750153,CS,BBG00NBK0GC3,True,2020-12-22,2023-12-29,2024-12-03,[Canoo Inc. Class A Common Stock],[BBG00NBK0H68],[],[usd],[us],[stocks]
GOEVW,XNAS,0001750153,WARRANT,BBG00NWRF3R2,True,2020-12-22,2023-12-29,2024-12-03,[Canoo Inc. Warrant],[],[],[usd],[us],[stocks]
GOEX,ARCX,0001432353,ETF,BBG0018HBWX2,True,2018-01-02,2023-12-29,2024-12-03,[Global X Gold Explorers ETF (NEW)],[BBG001TF8DG8],[],[usd],[us],[stocks]


In [9]:
ticker_universe[ticker_universe.index.get_level_values('ticker').str.contains("TEST")].sort_index()

,,,,,,start_date,end_date,last_updated_utc,name,share_class_figi,delisted_utc,currency_name,locale,market
ticker,primary_exchange,cik,type,composite_figi,active,,,,,,,,,
ATEST,XASE,0001415311,NaN,BBG000Q2HK03,False,2018-01-02,2023-12-29,2016-10-25,[None],[BBG001S6X7W6],[2016-10-25],[usd],[us],[stocks]
ATEST.A,XASE,NaN,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
ATEST.B,XASE,0000849998,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
ATEST.C,XASE,0000860188,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
NTEST,XNYS,0001515980,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
NTEST.A,XNYS,0001566897,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
NTEST.B,XNYS,0000807884,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
NTEST.C,XNYS,0000805264,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]


In [10]:
valid_tickers = pa.array([ticker for ticker in ticker_universe.index.get_level_values('ticker').to_list() if "TEST" not in ticker])
len(valid_tickers)

49781

In [11]:
def get_valid_tickers(config: PolygonConfig):
    tickers = get_ticker_universe(config)
    return pa.array([ticker for ticker in tickers.index.get_level_values('ticker').to_list() if "TEST" not in ticker])

valid_tickers = get_valid_tickers(config)
len(valid_tickers)

tickers_csv_path='/home/jovyan/work/data/tickers/tickers_2018-01-01_2023-12-31.csv'
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 49789 entries, ('CpP', 'XNYS', '0000831001', 'PFD', nan, True) to ('ZZrw', 'XNYS', '0000748015', 'RIGHT', nan, False)
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   start_date        49789 non-null  datetime64[ns]
 1   end_date          49789 non-null  datetime64[ns]
 2   last_updated_utc  49789 non-null  object        
 3   name              49789 non-null  object        
 4   share_class_figi  49789 non-null  object        
 5   delisted_utc      49789 non-null  object        
 6   currency_name     49789 non-null  object        
 7   locale            49789 non-null  object        
 8   market            49789 non-null  object        
dtypes: datetime64[ns](2), object(7)
memory usage: 6.3+ MB


49781

In [7]:
def iterate_all_aggs_tables(config: PolygonConfig, start_session: str = "pre", end_session: str = "market_open"):
    calendar = pandas_market_calendars.get_calendar(config.calendar_name)
    schedule = calendar.schedule(start_date=config.start_date,
                                 end_date=config.end_date,
                                 start="pre",
                                 end="post")
    for date, sessions in schedule.iterrows():
        # print(f"{date=} {sessions=}")
        start_dt = sessions[start_session]
        end_dt = sessions[end_session]
        # print(f"{date=} {start_dt=} {end_dt=}")
        aggs_ds = pa_ds.dataset(config.custom_aggs_dir,
                                format="parquet",
                                schema=custom_aggs_schema(),
                                partitioning=custom_aggs_partitioning())
        date_filter_expr = ((pc.field('year') == date.year)
                       & (pc.field('month') == date.month)
                       & (pc.field('date') == date.to_pydatetime().date()))
        # print(f"{date_filter_expr=}")
        for fragment in aggs_ds.get_fragments(filter=date_filter_expr):
            session_filter = ((pc.field('window_start') >= start_dt)
                              & (pc.field('window_start') < end_dt)
                              & pc.is_in(pc.field('ticker'), valid_tickers)
                             )
            # Sorting table doesn't seem to avoid needing to sort the df.  Maybe use_threads=False on to_pandas would help?
            # table = fragment.to_table(filter=session_filter).sort_by([('ticker', 'ascending'), ('window_start', 'descending')])
            table = fragment.to_table(filter=session_filter)
            if table.num_rows > 0:
                metadata = dict(table.schema.metadata) if table.schema.metadata else dict()
                metadata["date"] = date.date().isoformat()
                table = table.replace_schema_metadata(metadata)
                yield table

In [8]:
%%time

config.start_date = '2022-9-01'
config.end_date = '2022-12-31'

premarket_table = pa.concat_tables(iterate_all_aggs_tables(config))
premarket_table

CPU times: user 2.7 s, sys: 1.25 s, total: 3.95 s
Wall time: 2.73 s


pyarrow.Table
ticker: string not null
window_start: timestamp[ns, tz=UTC]
open: double
high: double
low: double
close: double
total: double
volume: int64
transactions: int64
vwap: double
----
ticker: [["AA","AAL","AAP","AAPL","AB",...,"KO","KOLD","KORU","KRBN","KRBP"],["LABD","LABU","LAC","LGMK","LI",...,"HYG","HYT","ICLN","IEUR","IJH"],...,["AA","AAL","AAON","AAPL","ABB",...,"AVCT","AVXL","AZEK","BABA","BBBY"],["BBIG","BILI","BLU","BNGO","BNKD",...,"ZIVO","ZKIN","ZM","ZROZ","ZYXI"]]
window_start: [[2022-09-01 08:00:00.000000000Z,2022-09-01 08:00:00.000000000Z,2022-09-01 08:00:00.000000000Z,2022-09-01 08:00:00.000000000Z,2022-09-01 08:00:00.000000000Z,...,2022-09-01 10:53:00.000000000Z,2022-09-01 10:53:00.000000000Z,2022-09-01 10:53:00.000000000Z,2022-09-01 10:53:00.000000000Z,2022-09-01 10:53:00.000000000Z],[2022-09-01 10:53:00.000000000Z,2022-09-01 10:53:00.000000000Z,2022-09-01 10:53:00.000000000Z,2022-09-01 10:53:00.000000000Z,2022-09-01 10:53:00.000000000Z,...,2022-09-01 12:32:00.

In [9]:
premarket_table.schema.metadata

{b'date': b'2022-09-01'}

In [10]:
premarket_table.nbytes

546423116

In [11]:
df = premarket_table.to_pandas()
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6763271 entries, 0 to 6763270
Data columns (total 10 columns):
 #   Column        Dtype              
---  ------        -----              
 0   ticker        object             
 1   window_start  datetime64[ns, UTC]
 2   open          float64            
 3   high          float64            
 4   low           float64            
 5   close         float64            
 6   total         float64            
 7   volume        int64              
 8   transactions  int64              
 9   vwap          float64            
dtypes: datetime64[ns, UTC](1), float64(6), int64(2), object(1)
memory usage: 516.0+ MB


,ticker,window_start,open,high,low,close,total,volume,transactions,vwap
0,AA,2022-09-01 08:00:00+00:00,48.88,48.88,48.88,48.88,7332.00,150,1,48.880000
1,AAL,2022-09-01 08:00:00+00:00,12.98,12.98,12.90,12.90,74935.39,5807,16,12.904321
2,AAP,2022-09-01 08:00:00+00:00,168.00,168.00,168.00,168.00,672.00,4,1,168.000000
3,AAPL,2022-09-01 08:00:00+00:00,156.00,156.42,155.88,156.27,473065.69,3030,141,156.127290
4,AB,2022-09-01 08:00:00+00:00,42.94,42.94,42.94,42.94,729.98,17,1,42.940000
...,...,...,...,...,...,...,...,...,...,...
6763266,ZIVO,2022-12-30 14:29:00+00:00,2.45,2.45,2.43,2.43,244.00,100,2,2.440000
6763267,ZKIN,2022-12-30 14:29:00+00:00,0.42,0.42,0.42,0.42,210.00,500,3,0.420000
6763268,ZM,2022-12-30 14:29:00+00:00,66.77,66.83,66.75,66.83,1469.34,22,3,66.788182
6763269,ZROZ,2022-12-30 14:29:00+00:00,87.64,87.64,87.64,87.64,4382.00,50,1,87.640000


In [12]:
table = next(iterate_all_aggs_tables(config))
table.schema.metadata

{b'date': b'2022-09-01'}

In [13]:
aggs_tables_iterator = iterate_all_aggs_tables(config)
next(aggs_tables_iterator)
table = next(aggs_tables_iterator)
table.schema.metadata

{b'date': b'2022-09-02'}

In [14]:
table.schema

ticker: string not null
window_start: timestamp[ns, tz=UTC]
open: double
high: double
low: double
close: double
total: double
volume: int64
transactions: int64
vwap: double
-- schema metadata --
date: '2022-09-02'

In [60]:
# https://github.com/twopirllc/pandas-ta/issues/731#issuecomment-1766786952

# def calculate_mfi(high, low, close, volume, period):
#     typical_price = (high + low + close) / 3
#     money_flow = typical_price * volume
#     mf_sign = np.where(typical_price > np.roll(typical_price, shift=1), 1, -1)
#     signed_mf = money_flow * mf_sign

#     # Calculate gain and loss using vectorized operations
#     positive_mf = np.maximum(signed_mf, 0)
#     negative_mf = np.maximum(-signed_mf, 0)

#     mf_avg_gain = np.convolve(positive_mf, np.ones(period), mode='full')[:len(positive_mf)] / period
#     mf_avg_loss = np.convolve(negative_mf, np.ones(period), mode='full')[:len(negative_mf)] / period

#     epsilon = 1e-10  # Small epsilon value to avoid division by zero
#     mfi = 100 - 100 / (1 + mf_avg_gain / (mf_avg_loss + epsilon))
#     return mfi

def calculate_mfi(typical_price: pd.Series, money_flow: pd.Series, period: int):
    mf_sign = np.where(typical_price > np.roll(typical_price, shift=1), 1, -1)
    signed_mf = money_flow * mf_sign

    # Calculate gain and loss using vectorized operations
    positive_mf = np.maximum(signed_mf, 0)
    negative_mf = np.maximum(-signed_mf, 0)

    mf_avg_gain = np.convolve(positive_mf, np.ones(period), mode='full')[:len(positive_mf)] / period
    mf_avg_loss = np.convolve(negative_mf, np.ones(period), mode='full')[:len(negative_mf)] / period

    epsilon = 1e-10  # Small epsilon value to avoid division by zero
    mfi = 100 - (100 / (1 + mf_avg_gain / (mf_avg_loss + epsilon)))
    return mfi

In [61]:
# https://github.com/twopirllc/pandas-ta/blob/main/pandas_ta/momentum/stoch.py
# https://github.com/twopirllc/pandas-ta/blob/development/pandas_ta/momentum/stoch.py
# `k` vs `fast_k` arg names.
# https://github.com/twopirllc/pandas-ta/issues/726
# Results affected by values outside range
# https://github.com/twopirllc/pandas-ta/issues/535

def calculate_stoch(high: pd.Series, low: pd.Series, close: pd.Series, k: int = 14, d: int = 3, smooth_k: int = 3, mamode:str = "sma"):
    """Indicator: Stochastic Oscillator (STOCH)"""
    lowest_low = low.rolling(k).min()
    highest_high = high.rolling(k).max()

    stoch = 100 * (close - lowest_low)
    stoch /= ta.utils.non_zero_range(highest_high, lowest_low)

    stoch_k = ta.overlap.ma(mamode, stoch.loc[stoch.first_valid_index():,], length=smooth_k)
    stoch_d = ta.overlap.ma(mamode, stoch_k.loc[stoch_k.first_valid_index():,], length=d) if stoch_k is not None else None
    # Histogram
    stoch_h = stoch_k - stoch_d if stoch_d is not None else None

    return stoch_k, stoch_d, stoch_h

In [99]:
def compute_per_ticker_signals(df: pd.DataFrame, period: int = 14) -> pd.DataFrame:
    df = df.set_index('window_start').sort_index()
    session_index = pd.date_range(start=df.index[0],
                                  end=df.index[-1],
                                  freq=pd.Timedelta(seconds=60))
    df = df.reindex(session_index)

    # df["minute_of_day"] = (df.index.hour * 60) + df.index.minute
    # df["day_of_week"] = df.index.day_of_week

    df.transactions = df.transactions.fillna(0)
    df.volume = df.volume.fillna(0)
    df.total = df.total.fillna(0)
    df.close = df.close.ffill()
    close = df.close
    df.vwap = df.vwap.fillna(close)
    df.high = df.high.fillna(close)
    df.low = df.low.fillna(close)
    df.open = df.open.fillna(close)
    price_open = df.open
    high = df.high
    low = df.low
    vwap = df.vwap
    # volume = df.volume
    total = df.total
    next_close = close.shift()

    # TODO: Odometer rollover signal.  Relative difference to nearest power of 10.
    # Something about log10 being a whole number?  When is $50 the rollover vs $100 or $10?

    # "True (Typical?) Price" which I think is an approximation of VWAP.
    # Trouble with both is that if there are no trades in a bar we get NaN.
    # That then means we get NaN for averages for the next period-1 bars too.
    # Question is whether to ffill the price for these calculations.
    df["TP"] = (high + low + close) / 3

    # Gain/loss in this bar.
    df["ret1bar"] = close.div(price_open).sub(1)

    for t in range(2, period):
        df[f'ret{t}bar'] = close.div(price_open.shift(t-1)).sub(1)

    # Average True Range (ATR)
    true_range = pd.concat([high.sub(low),
                            high.sub(next_close).abs(),
                            low.sub(next_close).abs()], axis=1).max(1)
    # Normalized ATR (NATR) or Average of Normalized TR.
    # Choice of NATR operations ordering discussion: https://www.macroption.com/normalized-atr/
    # He doesn't talk about VWAP but I think that is a better normalizing price for a bar.
    # atr = true_range.ewm(span=period).mean()
    # df["natr_c"] = atr / close
    # df["antr_c"] = (true_range / close).ewm(span=period).mean()
    # df["natr_v"] = atr / vwap
    # df["antr_v"] = (true_range / vwap).ewm(span=period).mean()
    df["NATR"] = (true_range / vwap).ewm(span=period).mean()

    # True Price as HLC average VS VWAP.
    # VWAP is better I think but is quite different than standard CCI.
    # Three ways to compute CCI, all give the same value using TP.
    # tp = (high + low + close) / 3
    # df['SMA'] = ta.sma(tp, length=period)
    # df['sma_r'] = tp.rolling(period).mean()
    # df['MAD'] = ta.mad(tp, length=period)
    # # Series.mad deprecated. mad = (s - s.mean()).abs().mean()
    # df['mad_r'] = tp.rolling(period).apply(lambda x: (pd.Series(x) - pd.Series(x).mean()).abs().mean())

    # df['cci_r'] = (tp - df['sma_r']) / (0.015 * df['mad_r'])
    # df['CCI'] = (tp - df['SMA']) / (0.015 * df['MAD'])
    # df['cci_ta'] = ta.cci(high=high, low=low, close=close, length=period)

    df['taCCI'] = ta.cci(high=high, low=low, close=close, length=period)

    # https://gist.github.com/quantra-go-algo/1b37bfb74d69148f0dfbdb5a2c7bdb25
    # https://medium.com/@huzaifazahoor654/how-to-calculate-cci-in-python-a-step-by-step-guide-9a3f61698be6
    sma = pd.Series(ta.sma(vwap, length=period))
    mad = pd.Series(ta.mad(vwap, length=period))
    df['CCI'] = (vwap - sma) / (0.015 * mad)

    # df['MFI'] = calculate_mfi(high=high, low=low, close=close, volume=volume, period=period)
    df['MFI'] = calculate_mfi(typical_price=vwap, money_flow=total, period=period)

    # We use Stochastic (rather than MACD because we need a ticker independent indicator.
    # IOW a percentage price oscillator (PPO) rather than absolute price oscillator (APO).
    # https://www.alpharithms.com/moving-average-convergence-divergence-macd-031217/
    # We're using 14/3 currently rather than the usual 26/12 popular for MACD though.
    stoch_k, stoch_d, stoch_h = calculate_stoch(high, low, close, k=period)
    df["STOCHk"] = stoch_k
    df["STOCHd"] = stoch_d
    df["STOCHh"] = stoch_h

    return df

In [98]:
%%time

df = table.to_pandas()
signals_df = df.groupby("ticker").apply(compute_per_ticker_signals, include_groups=False)
signals_df

<timed exec>:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


CPU times: user 30.8 s, sys: 33 ms, total: 30.9 s
Wall time: 30.9 s


open    high     low   close    total  volume  transactions        vwap          TP   ret1bar  ...  ret11bar  ret12bar  ret13bar      NATR       taCCI         CCI  MFI  STOCHk  STOCHd  STOCHh
ticker                                                                                                                             ...                                                                                             
A      2022-09-02 12:30:00+00:00  128.93  128.93  128.93  128.93  1676.09    13.0           1.0  128.930000  128.930000  0.000000  ...       NaN       NaN       NaN  0.000000         NaN         NaN  0.0     NaN     NaN     NaN
AA     2022-09-02 08:42:00+00:00   48.72   48.72   48.72   48.72    48.72     1.0           1.0   48.720000   48.720000  0.000000  ...       NaN       NaN       NaN  0.000000         NaN         NaN  0.0     NaN     NaN     NaN
       2022-09-02 08:43:00+00:00   48.72   48.72   48.72   48.72     0.00     0.0           0.0   48.720000   48.720000  0.000000  ...       NaN       NaN       NaN  0.000000         NaN         NaN  0.0     NaN     NaN     NaN
       2022-09-02 08:44:00+00:00   48.72   48.72   48.72   48.72     0.00     0.0           0.0   48.720000   48.720000  0.000000  ...       NaN       NaN       NaN  0.000000         NaN         NaN  0.0     NaN     NaN     NaN
       2022-09-02 08:45:00+00:00   48.72   48.72   48.72   48.72     0.00     0.0           0.0   48.720000   48.720000  0.000000  ...       NaN       NaN       NaN  0.000000         NaN         NaN  0.0     NaN     NaN     NaN
...                                  ...     ...     ...     ...      ...     ...           ...         ...         ...       ...  ...       ...       ...       ...       ...         ...         ...  ...     ...     ...     ...
ZYXI   2022-09-02 13:21:00+00:00    9.09    9.09    9.09    9.09     0.00     0.0           0.0    9.090000    9.090000  0.000000  ...    0.0000    0.0000    0.0000  0.000000    0.000000    0.000000  0.0     0.0     0.0     0.0
       2022-09-02 13:22:00+00:00    9.09    9.09    9.09    9.09     0.00     0.0           0.0    9.090000    9.090000  0.000000  ...    0.0000    0.0000    0.0000  0.000000    0.000000    0.000000  0.0     0.0     0.0     0.0
       2022-09-02 13:23:00+00:00    9.09    9.09    9.09    9.09     0.00     0.0           0.0    9.090000    9.090000  0.000000  ...    0.0000    0.0000    0.0000  0.000000    0.000000    0.000000  0.0     0.0     0.0     0.0
       2022-09-02 13:24:00+00:00    9.09    9.09    9.09    9.09     0.00     0.0           0.0    9.090000    9.090000  0.000000  ...    0.0000    0.0000    0.0000  0.000000    0.000000    0.000000  0.0     0.0     0.0     0.0
       2022-09-02 13:25:00+00:00    9.08    9.08    9.06    9.06   798.32    88.0           2.0    9.071818    9.066667 -0.002203  ...   -0.0033   -0.0033   -0.0033  0.000441 -466.666667 -466.666667  0.0     0.0     0.0     0.0

[536240 rows x 29 columns]

In [100]:
show(signals_df.loc["AAPL"])

In [101]:
signals_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 536240 entries, ('A', Timestamp('2022-09-02 12:30:00+0000', tz='UTC')) to ('ZYXI', Timestamp('2022-09-02 13:25:00+0000', tz='UTC'))
Data columns (total 29 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   open          536240 non-null  float64
 1   high          536240 non-null  float64
 2   low           536240 non-null  float64
 3   close         536240 non-null  float64
 4   total         536240 non-null  float64
 5   volume        536240 non-null  float64
 6   transactions  536240 non-null  float64
 7   vwap          536240 non-null  float64
 8   TP            536240 non-null  float64
 9   ret1bar       536240 non-null  float64
 10  ret2bar       531920 non-null  float64
 11  ret3bar       528955 non-null  float64
 12  ret4bar       526037 non-null  float64
 13  ret5bar       523146 non-null  float64
 14  ret6bar       520273 non-null  float64
 15  ret7bar       517437 non-null  f

In [102]:
def iterate_all_aggs_with_signals(config: PolygonConfig):
    for table in iterate_all_aggs_tables(config):
        df = table.to_pandas()
        df = df.groupby("ticker").apply(compute_per_ticker_signals, include_groups=False)
        yield pa.Table.from_pandas(df)



In [13]:
def find_price_runs(df: pd.DataFrame, min_price: float = 0.5, min_gain: float = 15, min_traded_value: float = 100000, min_duration: int = 10):
    df = df.set_index('window_start').sort_index()
    session_index = pd.date_range(start=df.index[0],
                                  end=df.index[-1],
                                  freq=pd.Timedelta(seconds=60))
    df = df.reindex(session_index)    
    prices = df['vwap'].values
    diffs = np.diff(prices)
    change_points = np.where(diffs < 0)[0] + 1

    # Add start and end points
    change_points = np.concatenate(([0], change_points, [len(prices)]))

    runs = []
    for i in range(len(change_points) - 1):
        start_idx = change_points[i]
        end_idx = change_points[i + 1] - 1
        if end_idx >= start_idx + 3:
            start_price = prices[start_idx]
            end_price = prices[end_idx]
            gain = 100 * (end_price - start_price) / start_price
            if (gain > min_gain) & (start_price > min_price):
                # TODO: Return up to ten prices and resample if longer to fit.
                start_dt = df.index[start_idx]
                end_dt = df.index[end_idx]
                price_series = df.loc[start_dt:end_dt]['vwap']
                volume_series = df.loc[start_dt:end_dt]["volume"]
                volume = volume_series.sum()
                traded_value = (price_series * volume_series).sum()
                if traded_value > min_traded_value:
                    previous_price = prices[start_idx-1] if (start_idx > 0) else None
                    next_price = prices[end_idx+1] if (end_idx + 1 < len(prices)) else None
                    if len(price_series) > min_duration:
                        price_series = price_series.iloc[::(len(price_series) // min_duration)]
                    runs.append({"gain": gain,
                                 "duration": end_idx - start_idx,
                                 "volume": volume,
                                 "traded_value": traded_value,
                                 "start_dt": start_dt,
                                 "end_dt": end_dt,
                                 "previous_price": previous_price,
                                 "start_price": start_price,
                                 "end_price": end_price,
                                 "next_price": next_price,
                                 "prices": price_series.round(3).to_list()
                                })
    if len(runs) > 0:
        return pd.DataFrame(runs).set_index("start_dt").sort_index()
    # This doesn't help with the empty warning.
    # return pd.DataFrame(data=[],
    #                     index=pd.DatetimeIndex([], name='start_dt'))
    return None


def label_price_runs_by_ticker(df: pd.DataFrame):
    # Ignoring this warning from the groupby call.
    # Tried suggested fixes but haven't figured out one that works.
    # FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. 
    # In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes.
    # To retain the old behavior, exclude the relevant entries before the concat operation.
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning)
        return df.set_index('ticker').sort_index().groupby(level=0).apply(find_price_runs, include_groups=False)


def label_price_runs_ds(config: PolygonConfig, valid_tickers: pa.Array):
    calendar = pandas_market_calendars.get_calendar(config.calendar_name)
    schedule = calendar.schedule(start_date=config.start_date,
                                 end_date=config.end_date,
                                 start="pre",
                                 end="post")
    labeled_aggs_dfs = []
    for date, sessions in schedule.iterrows():
        # print(f"{date=} {sessions=}")
        start_dt = sessions['pre']
        end_dt = sessions['market_open']
        print(f"{date=} {start_dt=} {end_dt=}")
        aggs_ds = pa_ds.dataset(config.custom_aggs_dir,
                                format="parquet",
                                schema=custom_aggs_schema(),
                                partitioning=custom_aggs_partitioning())
        date_filter_expr = ((pc.field('year') == date.year)
                       & (pc.field('month') == date.month)
                       & (pc.field('date') == date.to_pydatetime().date()))
        # print(f"{date_filter_expr=}")
        for fragment in aggs_ds.get_fragments(filter=date_filter_expr):
            session_filter = ((pc.field('window_start') >= start_dt)
                              & (pc.field('window_start') < end_dt)
                              & pc.is_in(pc.field('ticker'), valid_tickers)
                             )
            # Sorting table doesn't seem to avoid needing to sort the df.  Maybe use_threads=False on to_pandas would help?
            # table = fragment.to_table(filter=session_filter).sort_by([('ticker', 'ascending'), ('window_start', 'descending')])
            table = fragment.to_table(filter=session_filter)
            if table.num_rows > 0:
                days_runs_df = label_price_runs_by_ticker(table.to_pandas())
                if days_runs_df is not None:
                    days_runs_df.dropna(inplace=True)
                    if len(days_runs_df) > 0:
                        labeled_aggs_dfs.append(days_runs_df)

    return pd.concat(labeled_aggs_dfs)

In [14]:
%%time

config.start_date = '2024-12-01'
config.end_date = '2024-12-07'
price_run_dfs = label_price_runs_ds(config, valid_tickers)
show(price_run_dfs)

date=Timestamp('2024-12-02 00:00:00') start_dt=Timestamp('2024-12-02 09:00:00+0000', tz='UTC') end_dt=Timestamp('2024-12-02 14:30:00+0000', tz='UTC')
date=Timestamp('2024-12-03 00:00:00') start_dt=Timestamp('2024-12-03 09:00:00+0000', tz='UTC') end_dt=Timestamp('2024-12-03 14:30:00+0000', tz='UTC')
date=Timestamp('2024-12-04 00:00:00') start_dt=Timestamp('2024-12-04 09:00:00+0000', tz='UTC') end_dt=Timestamp('2024-12-04 14:30:00+0000', tz='UTC')
date=Timestamp('2024-12-05 00:00:00') start_dt=Timestamp('2024-12-05 09:00:00+0000', tz='UTC') end_dt=Timestamp('2024-12-05 14:30:00+0000', tz='UTC')
date=Timestamp('2024-12-06 00:00:00') start_dt=Timestamp('2024-12-06 09:00:00+0000', tz='UTC') end_dt=Timestamp('2024-12-06 14:30:00+0000', tz='UTC')


CPU times: user 13.6 s, sys: 267 ms, total: 13.8 s
Wall time: 13.6 s


In [9]:
price_run_dfs.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 103 entries, ('BREA', Timestamp('2024-12-02 11:38:00+0000', tz='UTC')) to ('UPC', Timestamp('2024-12-06 13:06:00+0000', tz='UTC'))
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gain            103 non-null    float64            
 1   duration        103 non-null    int64              
 2   volume          103 non-null    float64            
 3   traded_value    103 non-null    float64            
 4   end_dt          103 non-null    datetime64[ns, UTC]
 5   previous_price  103 non-null    float64            
 6   start_price     103 non-null    float64            
 7   end_price       103 non-null    float64            
 8   next_price      103 non-null    float64            
 9   prices          103 non-null    object             
dtypes: datetime64[ns, UTC](1), float64(7), int64(1), object(1)
memory usage: 329.9+ KB


In [10]:
pd.set_option('display.precision', 2)

In [11]:
%%time

config.start_date = '2022-9-01'
config.end_date = '2022-12-31'
runs_df = label_price_runs_ds(config)
show(runs_df)

date=Timestamp('2022-09-01 00:00:00') start_dt=Timestamp('2022-09-01 08:00:00+0000', tz='UTC') end_dt=Timestamp('2022-09-01 13:30:00+0000', tz='UTC')
date=Timestamp('2022-09-02 00:00:00') start_dt=Timestamp('2022-09-02 08:00:00+0000', tz='UTC') end_dt=Timestamp('2022-09-02 13:30:00+0000', tz='UTC')
date=Timestamp('2022-09-06 00:00:00') start_dt=Timestamp('2022-09-06 08:00:00+0000', tz='UTC') end_dt=Timestamp('2022-09-06 13:30:00+0000', tz='UTC')
date=Timestamp('2022-09-07 00:00:00') start_dt=Timestamp('2022-09-07 08:00:00+0000', tz='UTC') end_dt=Timestamp('2022-09-07 13:30:00+0000', tz='UTC')
date=Timestamp('2022-09-08 00:00:00') start_dt=Timestamp('2022-09-08 08:00:00+0000', tz='UTC') end_dt=Timestamp('2022-09-08 13:30:00+0000', tz='UTC')
date=Timestamp('2022-09-09 00:00:00') start_dt=Timestamp('2022-09-09 08:00:00+0000', tz='UTC') end_dt=Timestamp('2022-09-09 13:30:00+0000', tz='UTC')
date=Timestamp('2022-09-12 00:00:00') start_dt=Timestamp('2022-09-12 08:00:00+0000', tz='UTC') end_d

CPU times: user 2min 42s, sys: 1.96 s, total: 2min 44s
Wall time: 2min 42s


In [12]:
runs_df["time"] = runs_df.index.get_level_values("start_dt").time

In [13]:
show(runs_df[runs_df["prices"].apply(lambda p: not(any(pd.isna(x) for x in p[1:-2])))])

In [14]:
gapless_runs_df = runs_df[runs_df["prices"].apply(lambda p: not(any(pd.isna(x) for x in p[1:-2])))]
gapless_runs_df

gain  duration    volume  traded_value                    end_dt  previous_price  start_price  end_price  next_price                                             prices      time
ticker start_dt                                                                                                                                                                                                     
BGXX   2022-09-01 08:01:00+00:00  69.90         5  1.87e+05      3.92e+05 2022-09-01 08:06:00+00:00            1.43         1.43       2.43        2.12          [1.43, 1.517, 1.654, 2.001, 2.305, 2.429]  08:01:00
       2022-09-01 08:08:00+00:00  48.41         6  5.32e+05      1.32e+06 2022-09-01 08:14:00+00:00            2.12         2.02       3.00        2.95  [2.018, 2.162, 2.217, 2.237, 2.312, 2.459, 2.995]  08:08:00
HPCO   2022-09-01 09:35:00+00:00  16.15         7  1.83e+04      1.12e+05 2022-09-01 09:42:00+00:00            5.41         5.40       6.27        5.98  [5.4, 5.509, 5.681, 5.754, 5.765, 5.942, 6.181...  09:35:00
       2022-09-01 09:53:00+00:00  31.18         7  3.68e+04      2.65e+05 2022-09-01 10:00:00+00:00            5.93         5.88       7.71        7.54  [5.879, 5.95, 6.039, 6.272, 6.61, 7.007, 7.625...  09:53:00
       2022-09-01 10:32:00+00:00  23.24         6  1.31e+05      1.17e+06 2022-09-01 10:38:00+00:00            8.10         7.64       9.42        8.71  [7.642, 8.011, 8.118, 8.212, 8.941, 9.355, 9.418]  10:32:00
...                                 ...       ...       ...           ...                       ...             ...          ...        ...         ...                                                ...       ...
PALI   2022-12-30 11:55:00+00:00  36.18        14  6.90e+05      3.32e+06 2022-12-30 12:09:00+00:00            3.95         3.91       5.32        5.26  [3.909, 3.943, 3.946, 3.978, 4.084, 4.111, 4.2...  11:55:00
       2022-12-30 13:04:00+00:00  17.42         5  1.07e+06      7.15e+06 2022-12-30 13:09:00+00:00            6.05         6.04       7.09        6.92         [6.039, 6.242, 6.418, 6.604, 6.856, 7.091]  13:04:00
SILO   2022-12-30 13:16:00+00:00  31.47         4  4.41e+05      2.75e+06 2022-12-30 13:20:00+00:00            5.24         5.21       6.85        6.73                 [5.213, 5.666, 6.303, 6.44, 6.854]  13:16:00
TUEM   2022-12-30 10:21:00+00:00  27.58         4  1.00e+05      1.15e+05 2022-12-30 10:25:00+00:00            1.02         0.95       1.22        1.13                [0.953, 1.011, 1.071, 1.198, 1.216]  10:21:00
       2022-12-30 10:26:00+00:00  31.52         4  1.33e+05      1.74e+05 2022-12-30 10:30:00+00:00            1.22         1.13       1.48        1.48                [1.126, 1.223, 1.231, 1.243, 1.481]  10:26:00

[258 rows x 11 columns]

In [15]:
show(gapless_runs_df[gapless_runs_df["duration"] > 4])

In [16]:
runs_df.iloc[0]

gain                                                          24.38
duration                                                         14
volume                                                     203878.0
traded_value                                              191822.36
end_dt                                    2022-09-01 12:37:00+00:00
previous_price                                                  0.8
start_price                                                     0.8
end_price                                                       1.0
next_price                                                     0.96
prices            [0.8, 0.812, nan, nan, 0.82, nan, nan, nan, na...
time                                                       12:23:00
Name: (AKAN, 2022-09-01 12:23:00+00:00), dtype: object

In [ ]:
data['NATR'] = by_ticker.apply(lambda x: talib.NATR(x.high.ffill(),
                                                    x.low.ffill(),
                                                    x['last'].ffill()))